In [1]:
#import bibliotek
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, BasicAer)
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, array_to_latex
from qiskit.providers.aer import QasmSimulator


C:\Users\szymo\AppData\Local\Temp\ipykernel_7112\3498850633.py:2: DeprecationWarning: BasicAer is deprecated since Qiskit 0.46 and will be removed in Qiskit 1.0. The BasicAer (qiskit.providers.basicaer) module has been superseded by  qiskit.providers.basic_provider, and all its classes have been renamed to follow a new naming convention. More information and migration guidelines can be found in the 0.46 API docs for BasicAer.
  from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, BasicAer)
C:\Users\szymo\AppData\Local\Temp\ipykernel_7112\3498850633.py:6: DeprecationWarning: Importing from 'qiskit.providers.aer' is deprecated. Import from 'qiskit_aer' instead, which should work identically.
  from qiskit.providers.aer import QasmSimulator


# Kwantowa dystrybucja klucza BB84

Alice przesyła Bobowi klucz za pomocą wiązki fotonów w światłowodzie. Następnie Bob odcztyuje w zależności od bazy. Są dwie bazy : "z" = {|0>, |1>} oraz "x" = {|+>, |->}. Wykorzystuję się zjawisko polaryzacji światła

### Protokół BB84:

1. Alice losuję dwa ciągi binarne:

    a -> bity klucza
    
    b -> określenie bazy,
    
    Bob losuję jeden ciąg, który będzie jego bazami
    
    Ustalamy że:
    
        a = 0 -> |0>, |+>
        a = 1 -> |1>, |->
        b = 0 -> |0>, |1>
        b = 1 -> |+>, |->

In [2]:
qubits_number = 16
alice_a = np.random.randint(2, size=qubits_number)
alice_b = np.random.randint(2, size=qubits_number)
bob_b = np.random.randint(2, size=qubits_number)

print(f"Klucz Alice:\t {np.array2string(alice_a)}")
print(f"Baza Alice:\t {np.array2string(alice_b)}")
print(f"Baza Boba:\t {np.array2string(bob_b)}")

Klucz Alice:	 [0 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0]
Baza Alice:	 [1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1]
Baza Boba:	 [0 1 0 0 1 0 0 1 0 1 1 1 0 0 0 1]


2. Alice przygotowuję kubity zgodnie z wylosowanymi ciągami, wysyła je do Boba, a Bob w zależności od swoich wylosowanych baz odczytuje wynik 

In [3]:
bits_number = qubits_number
def bb84(key_bits, alice_base, bob_base):
    
    circuit = QuantumCircuit(qubits_number, bits_number)
    #Alice
    for i in range(len(alice_base)):
        if key_bits[i] == 1:
            circuit.x(i)
        if alice_base[i] == 1:
            circuit.h(i)
    #Bob        
    for j in range(len(bob_base)):
        if bob_base[j] == 1:
            circuit.h(j)
    
    circuit.measure(range(qubits_number),range(bits_number))
    
    return circuit

3. Alice i bob wymieniają się listą swoich baz, porównują je oraz wybierają tylko wyniki, które odpowiadają takim samym bazom

In [4]:
qc = bb84(alice_a, alice_b, bob_b)
simulator = Aer.get_backend('qasm_simulator')
bob_result = list(execute(qc.reverse_bits(),backend=simulator,shots=1).result().get_counts())[0]
#porównanie baz i utworzenie klucza
alice_key = ''
bob_key = ''
for i in range(qubits_number):
    if alice_b[i] == bob_b[i]:
        alice_key += str(alice_a[i])
        bob_key += str(bob_result[i])
print(f"Klucz Boba: {bob_key}")
print(f"Klucz Alice: {alice_key}")

differences = 0
for j in range(len(alice_key)):
    if alice_key[j] != bob_key[j]:
        differences += 1
print(f"Różnica w kluczach wynosi: {differences/len(alice_key)*100}%")

C:\Users\szymo\AppData\Local\Temp\ipykernel_7112\347923307.py:2: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  simulator = Aer.get_backend('qasm_simulator')
C:\Users\szymo\AppData\Local\Temp\ipykernel_7112\347923307.py:3: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  bob_result = list(execute(qc.reverse_bits(),backend=simulator,shots=1).result().get_counts())[0]


Klucz Boba: 011110
Klucz Alice: 011110
Różnica w kluczach wynosi: 0.0%


4. Widzimy że otrzymali taki sam klucz. Wiemy też, że prawdziwy świat nie jest tak idealny i jest możliwość różnicy jednak będzie ona dosyć niska. Sprawdźmy co się stanie gdy po drodze Ewa podsłucha wiadomość

In [5]:
# Ewa losuje swoją bazę
ewa_b = np.random.randint(2, size=qubits_number)
# następnie robi to co by zrobił Bob
wiretap_qc = bb84(alice_a, alice_b, ewa_b)
# wiadomość przechodzi do Boba
bob_circuit = QuantumCircuit(qubits_number, bits_number)
for j in range(len(bob_b)):
        if bob_b[j] == 1:
            bob_circuit.h(j)
bob_circuit.measure(range(qubits_number),range(bits_number))
wiretap_qc.compose(bob_circuit, inplace="true")
#odczyt i porównanie z kluczem Alice
simulator = Aer.get_backend('qasm_simulator')
bob_result = list(execute(wiretap_qc.reverse_bits(),backend=simulator,shots=1).result().get_counts())[0]
#porównanie baz i utworzenie klucza
alice_key = ''
bob_key = ''
for i in range(qubits_number):
    if alice_b[i] == bob_b[i]:
        alice_key += str(alice_a[i])
        bob_key += str(bob_result[i])
print(f"Klucz Boba: {bob_key}")
print(f"Klucz Alice: {alice_key}")

differences = 0
for j in range(len(alice_key)):
    if alice_key[j] != bob_key[j]:
        differences += 1
print(f"Różnica w kluczach wynosi: {differences/len(alice_key)*100}%")

Klucz Boba: 010010
Klucz Alice: 011110
Różnica w kluczach wynosi: 33.33333333333333%


C:\Users\szymo\AppData\Local\Temp\ipykernel_7112\4260808153.py:14: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  bob_result = list(execute(wiretap_qc.reverse_bits(),backend=simulator,shots=1).result().get_counts())[0]


Widzimy, że po odczycie Ewy kuibty mogły zmienić swój stan przez co klucz jaki otrzyma Bob pomimo porównania baz będzie się znacząco różnił od klucza Alice.
Dzięki takiemu zjawisku Bob i Alice będą wiedzieć, że ktoś po drodze podsłuchał ich kanał.